In [1]:
import json
import pennylane as qml
import pennylane.numpy as np
import scipy

In [87]:
def abs_dist(rho, sigma):
    """A function to compute the absolute value |rho - sigma|."""
    polar = scipy.linalg.polar(rho - sigma)
    return polar[1]

def word_dist(word):
    """A function which counts the non-identity operators in a Pauli word"""
    return sum(word[i] != "I" for i in range(len(word)))


# Produce the Pauli density for a given Pauli word and apply noise

def noisy_Pauli_density(word, lmbda):
    """
       A subcircuit which prepares a density matrix (I + P)/2**n for a given Pauli
       word P, and applies depolarizing noise to each qubit. Nothing is returned.

    Args:
            word (str): A Pauli word represented as a string with characters I,  X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.
    """


    # Put your code here #
    n = len(word)
    #print(n)
    P = qml.pauli.string_to_pauli_word(word)
    #print(P)
    I = qml.Identity([0])
    for i in range(1,n):
        I = I @ qml.Identity([i])
    rho_p = (P + I)*(0.5)**n
    rho_p_matrix = qml.matrix(rho_p, wire_order=range(n))

    qml.QubitDensityMatrix(rho_p_matrix, wires=range(n))
    for i in range(n):
        qml.DepolarizingChannel(lmbda, i)


# Compute the trace distance from a noisy Pauli density to the maximally mixed density

def maxmix_trace_dist(word, lmbda):
    """
       A function compute the trace distance between a noisy density matrix, specified
       by a Pauli word, and the maximally mixed matrix.

    Args:
            word (str): A Pauli word represented as a string with characters I, X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.

    Returns:
            float: The trace distance between two matrices encoding Pauli words.
    """
    n = len(word)
    dev = qml.device('default.mixed', wires=n)
    @qml.qnode(dev)
    def circuit():
    # Put your code here #
        noisy_Pauli_density(word, lmbda)
        return qml.density_matrix(wires=range(n))
    
    rho_p_matrix = circuit()
    
    I = qml.Identity([0])
    for i in range(1,n):
        I = I @ qml.Identity([i])

    rho_0 = (I)*(0.5)**n
    rho_0_matrix = qml.matrix(rho_0, wire_order=range(n))
    distance = abs_dist(rho_p_matrix, rho_0_matrix)

    return 0.5*np.trace(distance)

def bound_verifier(word, lmbda):
    """
       A simple check function which verifies the trace distance from a noisy Pauli density
       to the maximally mixed matrix is bounded by (1 - lambda)^|P|.

    Args:
            word (str): A Pauli word represented as a string with characters I, X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.

    Returns:
            float: The difference between (1 - lambda)^|P| and T(rho_P(lambda), rho_0).
    """


    # Put your code here #



    return (1-lmbda)**(word_dist(word)) - maxmix_trace_dist(word, lmbda)


# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:

    word, lmbda = json.loads(test_case_input)
    output = np.real(bound_verifier(word, lmbda))

    return str(output)


def check(solution_output: str, expected_output: str) -> None:

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your trace distance isn't quite right!"


test_cases = [['["XXI", 0.7]', '0.0877777777777777'], ['["XXIZ", 0.1]', '0.4035185185185055'], ['["YIZ", 0.3]', '0.30999999999999284'], ['["ZZZZZZZXXX", 0.1]', '0.22914458207245006']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '["XXI", 0.7]'...
Correct!
Running test case 1 with input '["XXIZ", 0.1]'...
Correct!
Running test case 2 with input '["YIZ", 0.3]'...
Correct!
Running test case 3 with input '["ZZZZZZZXXX", 0.1]'...
Correct!


------
# <center> Tests

In [41]:
def noisy_Pauli_density(word, lmbda):
    """
       A subcircuit which prepares a density matrix (I + P)/2**n for a given Pauli
       word P, and applies depolarizing noise to each qubit. Nothing is returned.

    Args:
            word (str): A Pauli word represented as a string with characters I,  X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.
    """


    # Put your code here #
    n = len(word)
    #print(n)
    P = qml.pauli.string_to_pauli_word(word)
    #print(P)
    I = qml.Identity([0])
    for i in range(1,n):
        I = I @ qml.Identity([i])
    rho_p = (P + I)*(0.5)**n
    
    for i in range(n):
        qml.DepolarizingChannel(lmbda, i)

In [71]:
word = "XXI"
n = len(word)
P = qml.pauli.string_to_pauli_word(word)
I = qml.Identity([0])
for i in range(1,n):
    I = I @ qml.Identity([i])
rho_p = (P + I)*(0.5)**n
#qml.ops.op_math.Sum(I, P)
#qml.ops.op_math.SProd(0.5**n, I)
rho_p

  (0.125) [I0]
+ (0.125) [X0 X1]


In [85]:
qml.matrix(P,wire_order=range(n)) - qml.matrix(qml.PauliX(0)@qml.PauliX(1)@qml.Identity(2))

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [81]:
qml.matrix(I+P, wire_order=range(n))

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [80]:
rho_p_matrix = qml.matrix(rho_p, wire_order=range(n))
rho_p_matrix.shape

(8, 8)

In [86]:
dev = qml.device('default.mixed', wires=n)
p = 1
@qml.qnode(dev)
def circuit():
    qml.QubitDensityMatrix(rho_p_matrix, wires=range(n))
    for i in range(n):
        qml.DepolarizingChannel(p, wires=i)
    return qml.density_matrix(wires=range(n))

circuit()

array([[0.125     +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.01388889+0.j, 0.        +0.j],
       [0.        +0.j, 0.125     +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.        +0.j, 0.01388889+0.j],
       [0.        +0.j, 0.        +0.j, 0.125     +0.j, 0.        +0.j,
        0.01388889+0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.125     +0.j,
        0.        +0.j, 0.01388889+0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.01388889+0.j, 0.        +0.j,
        0.125     +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j, 0.        +0.j, 0.01388889+0.j,
        0.        +0.j, 0.125     +0.j, 0.        +0.j, 0.        +0.j],
       [0.01388889+0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
        0.        +0.j, 0.        +0.j, 0.125     +0.j, 0.

# Resources:

https://discuss.pennylane.ai/t/using-mixed-states-as-input/2092
https://docs.pennylane.ai/en/stable/code/api/pennylane.matrix.html
https://docs.pennylane.ai/en/stable/code/api/pennylane.pauli.string_to_pauli_word.html
https://docs.pennylane.ai/en/stable/code/api/pennylane.density_matrix.html
https://docs.pennylane.ai/en/stable/code/api/pennylane.QubitDensityMatrix.html

